In [1]:
import arcpy
import os
pathname = r"C:\Users\admin\Desktop\School\GEOG 4057\Assignment 8"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname,"Austin.aprx"))
print(aprx.defaultGeodatabase)

C:\Users\admin\Desktop\School\GEOG 4057\Assignment 8\Austin_Data.gdb


In [2]:
aprx.saveACopy(os.path.join(pathname,'Austin_Copy.aprx'))


I opened Austin_Copy.aprx in ArcGIS Pro and confirmed that the maps, layers, and project contents looked the same as in the original Austin.aprx.

The original project file is 60 KB, while the copied project file is 47 KB. The file sizes are slightly different because ArcGIS Pro stores internal metadata and project history inside the APRX file. When I used the "saveACopy" it creates a new file and may rewrite or clean some of this internal information, which can change the size without changing the visible content of the project.

In [3]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
County
Foot_US


"del aprx" doesn’t delete the project. It just clears the aprx variable from Python so it’s not being used anymore. 

In [4]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

IndexError: list index out of range

In [5]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
Light Gray Reference
trees
parks
base
Light Gray Base
Map: County
facilities
hospitals
parks
Topographic


In [6]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


Light Gray Reference is a basemap layer
trees is a feature layer
parks is a feature layer
base is a feature layer
Light Gray Base is a basemap layer


In [7]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

In [8]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

The green variable is a python dictionary and the number sin the list refer to color values RGB and transparency. To make the layer brown I would use a color like [140, 70, 20, 100] instead.

In [9]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]

sym = lyr.symbology
brown = {"RGB": [140, 70, 20, 100]}  # brown color

if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = brown
    lyr.symbology = sym

In [15]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")

AttributeError: 'ArcGISProject' object has no attribute 'createLayout'

The size of the layout is 8.5 x 11 inches and the map frame size is 10 x 7.5 inches. Building the layout in Python is better because it gives exact sizes and placement without me having to drag things around. It’s also repeatable, so I can run the same code for different maps and get the same layout automatically. Overall it saves time and keeps everything consistent compared to doing it manually.

In [16]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

AttributeError: 'ArcGISProject' object has no attribute 'listStyleItems'

In [17]:
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element')
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'
leg.columnCount = 1
leg.title = 'Downtown'

AttributeError: 'ArcGISProject' object has no attribute 'listStyleItems'

In [19]:
lyt = aprx.listLayouts("New Layout with Rectangles")[0]

lyt.exportToPDF(os.path.join(pathname, 'downtown.pdf'))

'C:\\Users\\admin\\Desktop\\School\\GEOG 4057\\Assignment 8\\downtown.pdf'